In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
#!unzip preprocessed_images.zip -d ./preprocessed_images

In [3]:
import os
import pandas as pd
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image

#transform = transforms.Compose([
#    transforms.Resize((224, 224)),
#    transforms.RandomHorizontalFlip(),
#    transforms.RandomRotation(20),
#    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
#    transforms.ToTensor(),
#    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#])

#full_df = pd.read_csv("full_df.csv")

#print(full_df.head())
#full_df.columns


In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, Subset, WeightedRandomSampler
import numpy as np
from torchvision.transforms import ToTensor


class FundusDataset(Dataset):
    '''
    Custom Dataset para la carga de las imágenes de fondo de ojo con sus respectivos targets.
    Targets y sus significados:
    [1, 0, 0, 0, 0, 0, 0, 0] - Normal
    [0, 1, 0, 0, 0, 0, 0, 0] - Diabetes
    [0, 0, 1, 0, 0, 0, 0, 0] - Glaucoma
    [0, 0, 0, 1, 0, 0, 0, 0] - Cataratas
    [0, 0, 0, 0, 1, 0, 0, 0] - Degeneración macular asociada a la edad
    [0, 0, 0, 0, 0, 1, 0, 0] - Hipertensión
    [0, 0, 0, 0, 0, 0, 1, 0] - Miopía
    [0, 0, 0, 0, 0, 0, 0, 1] - Otras enfermedades o anormalidades
    '''
    def __init__(self, images_path, targets_df, transform=None):
        self.image_folder = ImageFolder(root=images_path, transform=transform)
        self.targets_df = targets_df
        self.to_tensor = ToTensor()

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img, _ = self.image_folder[idx]
        img = self.to_tensor(img)
        img_filename = os.path.basename(self.image_folder.imgs[idx][0])
        target = np.array(self._map_label_to_target(self.targets_df.loc[self.targets_df['filename'] == img_filename, 'labels'].values[0]))
        return img, target

    def _map_label_to_target(self, label):
        if label == '[\'N\']':
            return [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'D\']':
            return [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'G\']':
            return [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'C\']':
            return [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
        elif label == '[\'A\']':
            return [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
        elif label == '[\'H\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
        elif label == '[\'M\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
        elif label == '[\'O\']':
            return [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
        else:
            raise Exception('Target desconocido')

full_df = pd.read_csv("full_df.csv")

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset = FundusDataset(
    images_path='./preprocessed_images',
    targets_df=full_df)

# COMENTAR ESTAS LÍNEAS: Reducir el tamaño total del dataset a los efectos
# de realizar pruebas rápidas.
# dataset = Subset(dataset, range(300))
#train_dataset = FundusDataset(images_path='./preprocessed_images', targets_df=full_df, transform=train_transform)
#test_dataset = FundusDataset(images_path='./preprocessed_images', targets_df=full_df, transform=test_transform)

# Dividir el dataset en entrenamiento y testing.
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
#train_indices, val_indices = random_split(range(len(dataset)), [train_size, test_size])
train_dataset.transform = transform_train
#print(train_dataset.transform)
#train_subset.transform = transform_train
test_dataset.transform = transform_val
print(test_dataset.transform)

Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [5]:
# Subsampling
labels_counts = full_df['labels'].value_counts()
sample_weights = [
    1. / labels_counts['[\'N\']'],
    1. / labels_counts['[\'D\']'],
    1. / labels_counts['[\'G\']'],
    1. / labels_counts['[\'C\']'],
    1. / labels_counts['[\'A\']'],
    1. / labels_counts['[\'H\']'],
    1. / labels_counts['[\'M\']'],
    1. / labels_counts['[\'O\']']
]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(dataset), replacement=True)

# Instanciar dataloaders para ambos subsets.
train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Verificar shapes de los batches de entrenamiento.
train_features, train_labels = next(iter(train_dataloader))
print(f'Train feature batch shape: {train_features.size()}')
print(f'Train labels batch shape: {train_labels.size()}')

# Verificar shapes de los batches de testing.
test_features, test_labels = next(iter(test_dataloader))
print(f'Test feature batch shape: {test_features.size()}')
print(f'Test labels batch shape: {test_labels.size()}')

Train feature batch shape: torch.Size([8, 3, 512, 512])
Train labels batch shape: torch.Size([8, 8])
Test feature batch shape: torch.Size([8, 3, 512, 512])
Test labels batch shape: torch.Size([8, 8])


In [6]:
# Define the VGGNet model with dropout
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class VGGNetWithDropout(nn.Module):
    def __init__(self, num_classes):
        super(VGGNetWithDropout, self).__init__()
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        
        # Modify the classifier to include dropout
        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
    
    def forward(self, x):
        return self.vgg(x)


In [7]:


num_classes = 8

#model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
model = VGGNetWithDropout(num_classes)
#model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = nn.DataParallel(model, device_ids=[0, 1])
#model.to('cuda:0')
model = model.to(device)

print(model)


VGGNetWithDropout(
  (vgg): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

from torch.cuda.amp import autocast, GradScaler

# Training loop
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    scaler = GradScaler()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = torch.argmax(labels, dim=1)
        
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        #loss.backward()
        #optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        
        #predicted = (outputs > 0.5).float()
        _, predicted = torch.max(outputs, 1)
        #print(f'outputs shape: {outputs.shape}, predicted shape: {predicted.shape}, labels shape: {labels.shape}')
    
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        #correct += (predicted == labels).float().sum()
        #total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy


In [9]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            labels = torch.argmax(labels, dim=1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            #predicted = (outputs > 0.5).float()
            #correct += (predicted == labels).float().sum()
            #print(f'outputs shape: {outputs.shape}, predicted shape: {predicted.shape}, labels shape: {labels.shape}')
    
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    
    epoch_loss = running_loss / total
    epoch_accuracy = correct / total
    
    return epoch_loss, epoch_accuracy


In [10]:

num_epochs = 10

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_model(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate_model(model, test_dataloader, criterion, device)
    
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

torch.save(model.state_dict(), 'vgg16_odir.pth')


Epoch 1/10
Train Loss: 1.5158, Train Accuracy: 0.3772
Val Loss: 2.1977, Val Accuracy: 0.4574
Epoch 2/10
Train Loss: 1.3894, Train Accuracy: 0.3686
Val Loss: 2.1993, Val Accuracy: 0.4574
Epoch 3/10
Train Loss: 1.3959, Train Accuracy: 0.3578
Val Loss: 2.2344, Val Accuracy: 0.4574
Epoch 4/10
Train Loss: 1.3836, Train Accuracy: 0.3830
Val Loss: 2.3402, Val Accuracy: 0.4574
Epoch 5/10
Train Loss: 1.3822, Train Accuracy: 0.3742
Val Loss: 2.3249, Val Accuracy: 0.4574
Epoch 6/10
Train Loss: 1.3817, Train Accuracy: 0.3766
Val Loss: 2.3708, Val Accuracy: 0.4574
Epoch 7/10
Train Loss: 1.3777, Train Accuracy: 0.3619
Val Loss: 2.3656, Val Accuracy: 0.4574
Epoch 8/10
Train Loss: 1.3820, Train Accuracy: 0.3866
Val Loss: 2.3946, Val Accuracy: 0.4574
Epoch 9/10
Train Loss: 1.3808, Train Accuracy: 0.3656
Val Loss: 2.3833, Val Accuracy: 0.4574
Epoch 10/10
Train Loss: 1.3860, Train Accuracy: 0.3653
Val Loss: 2.5208, Val Accuracy: 0.4574
